<a href="https://colab.research.google.com/github/advik-7/Agents/blob/main/LangGraph_Custom_tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import langchain_core.tools
print(dir(langchain_core.tools))


['BaseTool', 'BaseToolkit', 'FILTERED_ARGS', 'InjectedToolArg', 'RetrieverInput', 'SchemaAnnotationError', 'StructuredTool', 'Tool', 'ToolException', 'ToolsRenderer', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_get_runnable_config_param', 'annotations', 'base', 'convert', 'convert_runnable_to_tool', 'create_retriever_tool', 'create_schema_from_function', 'render', 'render_text_description', 'render_text_description_and_args', 'retriever', 'simple', 'structured', 'tool']


In [17]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36


In [20]:
from transformers import pipeline
from langchain.prompts import PromptTemplate
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.memory import ConversationBufferMemory
import os


In [13]:
from langchain_core.tools import tool
import os

@tool
def create_file(file_name: str) -> str:
    """
    Creates a new file with the specified name.
    """
    try:
        with open(file_name, 'w') as f:
            return f"File '{file_name}' created successfully."
    except Exception as e:
        return f"Error creating file: {e}"

@tool
def write_to_file(file_name: str, content: str) -> str:
    """
    Writes content to a specified file, appending to the file if it exists.
    """
    try:
        with open(file_name, 'a') as f:
            f.write(content + '\n')
        return f"Content written to '{file_name}' successfully."
    except Exception as e:
        return f"Error writing to file: {e}"

@tool
def rename_file(old_name: str, new_name: str) -> str:
    """
    Renames an existing file from 'old_name' to 'new_name'.
    """
    try:
        os.rename(old_name, new_name)
        return f"File renamed from '{old_name}' to '{new_name}' successfully."
    except Exception as e:
        return f"Error renaming file: {e}"

@tool
def delete_file(file_name: str) -> str:
    """
    Deletes the specified file.
    """
    try:
        os.remove(file_name)
        return f"File '{file_name}' deleted successfully."
    except Exception as e:
        return f"Error deleting file: {e}"


In [21]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

tools = [
    create_file,
    write_to_file,
    rename_file,
    delete_file
]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [22]:


def llm_decision_maker(query: str) -> str:
    candidate_labels = ["create_file", "write_to_file", "rename_file", "delete_file"]

    result = classifier(query, candidate_labels)

    decision = result['labels'][0]
    return decision

class FileManagementAgent:
    def __init__(self):
        self.tools = {
            "create_file": create_file,
            "write_to_file": write_to_file,
            "rename_file": rename_file,
            "delete_file": delete_file
        }

    def handle_query(self, query: str):
        tool_decision = llm_decision_maker(query)
        if tool_decision not in self.tools:
            return "Unable to make a decision based on the query."

        tool = self.tools[tool_decision]

        if tool_decision == "create_file":
            return tool("example.txt")
        elif tool_decision == "write_to_file":
            return tool("example.txt", "This is some sample content.")
        elif tool_decision == "rename_file":
            return tool("example.txt", "new_example.txt")
        elif tool_decision == "delete_file":
            return tool("example.txt")


In [24]:

queries = [
    "Create a file named hehe.txt"
]

agent = FileManagementAgent()

for query in queries:
    print(f"Query: {query}")
    print(f"Response: {agent.handle_query(query)}\n")

Query: Create a file named hehe.txt
Response: File 'example.txt' created successfully.

